In [56]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity
import pickle

In [57]:
# Load the Excel file
file_path = 'DataSets/Places_Dataset_Test.xlsx'
df = pd.read_excel(file_path)

In [58]:
# Limit the dataset to the first 50 rows
df1 = df.iloc[:50, :]

# Display the first few rows of the limited dataset
df1.head()

,name,activities,lat,lng,formatted_address,rating,user_ratings_total,user_ratings_total.1
0,Arugam Bay Beach,"['beach visits', 'beachfront dining', 'surfing']",6.840408,81.836848,"Arugam Bay Beach, Sri Lanka",4.8,1591.0,['Arugam Bay Beach is a surfer's paradise! I s...
1,Mirissa Beach,"['beach visits', 'beachfront dining', 'snorkel...",5.944703,80.459161,"Mirissa, Sri Lanka",4.6,1748.0,['Mirissa Beach is truly a gem on Sri Lanka's ...
2,Weligama Beach (surf and stay),"['beach visits', 'surfing']",5.972486,80.435714,"Weligama, Sri Lanka",4.4,325.0,['Weligama Beach is a fantastic spot for both ...
3,Ahangama,"['beach visits', 'surfing']",5.973975,80.362159,"Ahangama, Sri Lanka",NaN,NaN,['Ahangama was a bit disappointing for me as a...
4,Hikkaduwa Beach,"['beach visits', 'snorkeling', 'surfing', 'tur...",6.137727,80.099060,"Hikkaduwa Beach, Sri Lanka",4.7,1438.0,['Hikkaduwa Beach is a delightful escape for s...


In [59]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   name                  50 non-null     object 
 1   activities            49 non-null     object 
 2   lat                   50 non-null     float64
 3   lng                   50 non-null     float64
 4   formatted_address     50 non-null     object 
 5   rating                41 non-null     float64
 6   user_ratings_total    41 non-null     float64
 7   user_ratings_total.1  50 non-null     object 
dtypes: float64(4), object(4)
memory usage: 3.3+ KB


In [60]:
# Drop rows that contain any null values
data = df1.dropna()
data.head()

,name,activities,lat,lng,formatted_address,rating,user_ratings_total,user_ratings_total.1
0,Arugam Bay Beach,"['beach visits', 'beachfront dining', 'surfing']",6.840408,81.836848,"Arugam Bay Beach, Sri Lanka",4.8,1591.0,['Arugam Bay Beach is a surfer's paradise! I s...
1,Mirissa Beach,"['beach visits', 'beachfront dining', 'snorkel...",5.944703,80.459161,"Mirissa, Sri Lanka",4.6,1748.0,['Mirissa Beach is truly a gem on Sri Lanka's ...
2,Weligama Beach (surf and stay),"['beach visits', 'surfing']",5.972486,80.435714,"Weligama, Sri Lanka",4.4,325.0,['Weligama Beach is a fantastic spot for both ...
4,Hikkaduwa Beach,"['beach visits', 'snorkeling', 'surfing', 'tur...",6.137727,80.099060,"Hikkaduwa Beach, Sri Lanka",4.7,1438.0,['Hikkaduwa Beach is a delightful escape for s...
6,Unawatuna Beach,"['feeding fish', 'paddleboarding', 'snorkeling']",6.009686,80.248424,"Unawatuna Beach, Sri Lanka",4.8,1868.0,['Unawatuna Beach is a slice of paradise! The ...


In [61]:
data.describe()

,lat,lng,rating,user_ratings_total
count,40.000000,40.000000,40.000000,40.000000
mean,6.939242,80.611912,4.505000,3386.325000
std,0.843526,0.482311,0.307137,5725.122573
min,5.944703,79.846480,3.700000,174.000000
25%,6.110910,80.234399,4.300000,386.250000
50%,6.809544,80.570662,4.550000,1157.500000
75%,7.668051,80.852582,4.800000,2733.500000
max,8.721837,81.836848,5.000000,26736.000000


In [ ]:
# structure of our dataset
# data = {
#     'name': ['Place A', 'Place B', 'Place C', 'Place D', 'Place E'],
#     'rating': [4.5, 3.9, 4.8, 4.2, 4.0],
#     'user_ratings_total': [200, 150, 180, 90, 130],
#     'activities': ['Hiking, Swimming', 'Beach', 'Hiking, Sightseeing', 'Beach, Hiking', 'Sightseeing, Swimming']
# }

places_df = pd.DataFrame(data)

# TF-IDF Vectorizer for Activities
tfidf = TfidfVectorizer(stop_words='english')
activities_matrix = tfidf.fit_transform(places_df['activities'])

# Normalize ratings and total ratings to bring them to the same scale
scaler = MinMaxScaler()
places_df[['Normalized_Average_Rating', 'Normalized_Total_Ratings']] = scaler.fit_transform(places_df[['rating', 'user_ratings_total']])

def recommend_places(preferred_activities, bucket_list_destination):
    # Add a boost to places that match the bucket list destination
    places_df['Bucket_List_Boost'] = places_df['name'].apply(lambda x: 10 if x == bucket_list_destination else 0)

    # Vectorize the user's activities
    user_activities_vector = tfidf.transform([preferred_activities])

    # Calculate the similarity between user preferences and place activities
    cosine_sim = cosine_similarity(user_activities_vector, activities_matrix)

    # Add similarity scores to the DataFrame
    places_df['Similarity_Score'] = cosine_sim[0]

    # Calculate the final score
    places_df['Final_Score'] = (
        (places_df['Similarity_Score'] * 0.5) + 
        (places_df['Normalized_Average_Rating'] * 0.2) + 
        (places_df['Bucket_List_Boost'] * 0.1) + 
        (places_df['Normalized_Total_Ratings'] * 0.2)
    )

    # Sort places based on the final score
    recommended_places = places_df.sort_values(by='Final_Score', ascending=False)

    # Return top 5 recommended places
    return recommended_places[['name', 'Final_Score']].head(5)

# Testing the model

In [64]:
# Example user inputs
preferred_activities = ''
bucket_list_destination = 'Puwakpitiya,Avissawella,Eheliyagoda'

# Call the function and display the recommended places
top_places = recommend_places(preferred_activities, bucket_list_destination)
print(top_places)

                           name  Final_Score
21          Sri Dalada Maligawa     0.369231
10                     Sigiriya     0.326171
8              Galle Dutch Fort     0.264657
30       Sri Pada / Adam's Peak     0.224191
17  Horton Plains National Park     0.217019


# Save model as pickle

In [50]:
# Save the TF-IDF vectorizer
with open('tfidf_model.pkl', 'wb') as file:
    pickle.dump(tfidf, file)

# Save the processed DataFrame (places_df)
with open('places_df.pkl', 'wb') as file:
    pickle.dump(places_df, file)

print("Models saved successfully.")


Models saved successfully.


In [ ]:
# Load the TF-IDF vectorizer
with open('tfidf_model.pkl', 'rb') as file:
    loaded_tfidf = pickle.load(file)

# Load the processed DataFrame (places_df)
with open('places_df.pkl', 'rb') as file:
    loaded_places_df = pickle.load(file)

print("Models loaded successfully.")
